In [ ]:
# mc 계정생성 - system 계정에서
create user mc identified by mc;
grant create session, create table, create view, create sequence to mc;
alter user mc default tablespace users;
alter user mc quota unlimited on users;

# 테이블 생성 - mc 계정에서
create table dbtest(
id varchar2(30) primary key,
passwd varchar2(30) not null,
name varchar(50) not null,
tel varchar(15),
address varchar2(100)
);

insert into dbtest values( 'aaa', '111', '홍길동', '1111-2222', '서울시' );
insert into dbtest values( 'bbb', '111', '이순신', '2222-2222', '수원시' );
insert into dbtest values( 'ccc', '111', '김유신', '3333-2222', '안산시' );
insert into dbtest values( 'ddd', '111', '대조영', '5555-2222', '인천시' );
insert into dbtest values( 'eee', '111', '강감찬', '6666-2222', '의정부시' );
commit;

select * from dbtest;

In [23]:
# select
import cx_Oracle
dsn = cx_Oracle.makedsn( 'localhost', '1521', 'xe' )
con = cx_Oracle.connect( 'mc', 'mc', dsn )
cursor = con.cursor()

sql = "select * from dbtest"
rows = cursor.execute( sql )

# print( type( rows ) )                       # cx_Oracle.Cursor
# for row in rows :                           # 반복문 실행 후 cursor가 맨 끝으로 이동
# #     print( type( row ) )                  # tuple
#     for column in row :
#         print( column, end=' ' )
#     print()        

users = cursor.fetchall()    
print( users[0][0] )
print( users[0] )                             # ('aaa', '111', '홍길동', '1111-2222', '서울시')

sql = "select count(*) from dbtest"
cursor.execute( sql )
count = cursor.fetchone()[0]
print( '회원수 : ', count )

sql = "select * from dbtest where id='aaa'"    
cursor.execute( sql )
# user = cursor.fetchall()                   # list
user = cursor.fetchone()                     # tuple    
print( user )
con.close()  

aaa
('aaa', '111', '홍길동', '1111-2222', '서울시')
회원수 :  5
('aaa', '111', '홍길동', '1111-2222', '서울시')


In [38]:
# insert / delete
import cx_Oracle
dsn = cx_Oracle.makedsn( 'localhost', '1521', 'xe' )
con = cx_Oracle.connect( 'mc', 'mc', dsn )
cursor = con.cursor()

# 한명
user = ( 'fff', '111', '안중근', '1111-3333', '일산시' )           # id primary 제약조건 위배 
sql = "insert into dbtest values(:1, :2, :3, :4, :5 )"
cursor.execute( sql, user )
con.commit()

# 여러명

# delete
sql = "select * from dbtest where id='fff'"
cursor.execute( sql )
user = cursor.fetchone()
if user == None :
    print( '입력한 아이디가 없습니다' )
else :
    if user[1] == '111' :
        sql = "delete from dbtest where id=:idx"
        cursor.execute( sql, idx='fff' )
        print( '탈퇴했습니다' )
    else :
        print( '입력한 비밀번호가 다릅니다' )
con.commit()

# 확인
sql = "select * from dbtest order by id"
rows = cursor.execute( sql )
for row in rows :
    for column in row :
        print( column, end=' ' )
    print()     

con.close()

탈퇴했습니다
aaa 111 홍길동 1111-2222 서울시 
bbb 111 이순신 2222-2222 수원시 
ccc 111 김유신 3333-2222 안산시 
ddd 111 대조영 5555-2222 인천시 
eee 111 강감찬 6666-2222 의정부시 


In [42]:
# update
import cx_Oracle
dsn = cx_Oracle.makedsn( 'localhost', '1521', 'xe' )
con = cx_Oracle.connect( 'mc', 'mc', dsn )
cursor = con.cursor()

idx = 'aaa'
passwd = '999'
tel = '9999-9999'
address = '평택시'

# sql = "update dbtest set passwd=:pd, tel=:t, address=:addr where id=:idx"
# cursor.execute( sql, pd=passwd, t=tel, addr=address, idx=idx )
sql = "update dbtest set passwd='"+passwd+"', tel='"+tel+"', address='"+address+"' where id='"+idx+"'"
cursor.execute( sql )
con.commit()
print( "수정됐습니다" )

# 확인
sql = "select * from dbtest order by id"
rows = cursor.execute( sql )
for row in rows :
    for column in row :
        print( column, end=' ' )
    print()     
con.close()

수정됐습니다
aaa 999 홍길동 9999-9999 평택시 
bbb 111 이순신 2222-2222 수원시 
ccc 111 김유신 3333-2222 안산시 
ddd 111 대조영 5555-2222 인천시 
eee 111 강감찬 6666-2222 의정부시 
